In [2]:
using Zygote
using LinearAlgebra

In [ ]:
gradient(x -> sum(x), [1, 2, 3])

In [ ]:
gradient(x -> sum(x) + x[end]^2, [1, 2, 3, 4, 5, 6])

In [ ]:
gradient((a, b) -> a*b, 2, 3)

In [ ]:
# simple MLP 3

W = rand(1, 3)
x = rand(3)

sigmoid = x -> 1/(1 + exp(-x))

gradient(W -> sigmoid(dot(W,x)), W)

In [ ]:
W*x

In [21]:
function f(x, Δt)
    position = x[1:3]
    quaternion = x[4:7]
    velocity = x[8:10]
    angular_vel = x[11:13]

    # quaternion update
    r = angular_vel
    mag = norm(r)

    sᵣ = cos(mag*Δt / 2.0)
    vᵣ = sin(mag*Δt / 2.0) * r/mag

    sₙ = quaternion[1]
    vₙ = quaternion[2:4]

    s = sₙ*sᵣ - vₙ'*vᵣ
    v = sₙ*vᵣ+sᵣ*vₙ+vₙ×vᵣ

    new_position = position + Δt * velocity
    new_quaternion = [s; v]
    return [new_position; new_quaternion; velocity; angular_vel]
end

function jac_fx(x, Δ)
    jacobian(state -> f(state, Δ), x)[1]
end

jac_fx (generic function with 1 method)

In [23]:
Δ = 0.1
x₀ = zeros(13)
x₀[3] = 1 # body off ground
x₀[7] = 1

x₀

13-element Vector{Float64}:
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [24]:
f(x₀, Δ)

13-element Vector{Float64}:
   0.0
   0.0
   1.0
 NaN
 NaN
 NaN
 NaN
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0

In [ ]:
jac_fx(x, Δ)

In [3]:
using Revise, VehicleSim

In [11]:
using Rotations

In [15]:
function get_imu_transform()
    R_imu_in_body = [0.9998 0.0 0.0199987;
                     0.0    1.0      0.0;
                     -0.0199987 0.0 0.9998]
    t_imu_in_body = [0, 0, 0.7]

    T = [R_imu_in_body t_imu_in_body]
end

get_imu_transform (generic function with 1 method)

In [12]:
RotY(0.02)

3×3 RotY{Float64} with indices SOneTo(3)×SOneTo(3)(0.02):
  0.9998     0.0  0.0199987
  0.0        1.0  0.0
 -0.0199987  0.0  0.9998

In [16]:
function h(x, output_gps_measurement=true)
    if output_gps_measurement
        # output in GPS frame
        T_body_to_gps = VehicleSim.get_gps_transform()
        gps_loc_body = T_body_to_gps*[zeros(3); 1.0]

        xyz_body = x[1:3]
        q_body = x[4:7]
        Tbody = VehicleSim.get_body_transform(q_body, xyz_body)
        xyz_gps = Tbody * [gps_loc_body; 1]
        return xyz_gps[1:2]
    else
        # output in IMU frame
        T_body_imu = get_imu_transform()
        T_imu_body = VehicleSim.invert_transform(T_body_imu)
        R = T_imu_body[1:3,1:3]
        p = T_imu_body[1:3,end]

        v_body = x[8:10]
        ω_body = x[11:13]

        ω_imu = R * ω_body
        v_imu = R * v_body + p × ω_imu

        return [v_imu; ω_imu]
    end
end

function jac_hx(x, output_gps_measurement)
    jacobian(state -> h(state, output_gps_measurement), x)
end

jac_hx (generic function with 1 method)

In [17]:
h(x, true)

2-element Vector{Float64}:
  11.4
 -11.0

In [18]:
h(x, false)

6-element Vector{Float64}:
 1.6796613201530972
 0.29999999160118307
 1.0337978024038263
 0.9798013117559599
 1.0
 1.019798712235861

In [19]:
jac_hx(x, true)[1]

2×13 Matrix{Float64}:
 1.0  0.0  0.0  -2.8    1.2  13.2   9.2  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  -9.2  -13.2   1.2  -2.8  0.0  0.0  0.0  0.0  0.0  0.0

In [20]:
jac_hx(x, false)[1]

6×13 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  …   0.0        0.69986     0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     -0.7        0.0         0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0      0.0        0.0139991   0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0      0.9998     0.0        -0.0199987
 0.0  0.0  0.0  0.0  0.0  0.0  0.0      0.0        1.0         0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  …   0.0199987  0.0         0.9998